In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Sat Dec 14 16:38:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# **Exploratory Data Analysis of Bitcoin BlockChain DATASET**

This dataset is designed for research on blockchain anomaly and fraud detection, created from Bitcoin network transaction data and donated to the Kaggle community. It includes metadata extracted from a directed-acyclic graph (DAG) of Bitcoin transactions.

Features:
- tx_hash: Hash of the Bitcoin transaction.
- indegree: Number of transactions that are inputs to tx_hash.
- outdegree: Number of transactions that are outputs from tx_hash.
- in_btc: Bitcoins on each incoming edge to tx_hash.
- out_btc: Bitcoins on each outgoing edge from tx_hash.
- total_btc: Net bitcoins flowing in and out from tx_hash.
- mean_in_btc: Average bitcoins flowing in for tx_hash.
- mean_out_btc: Average bitcoins flowing out for tx_hash.
- in-malicious: 1 if tx_hash is an input of a malicious transaction.
- out-malicious: 1 if tx_hash is an output of a malicious transaction.
- is-malicious: 1 if tx_hash is a malicious transaction (based on Bitcoin Hacks dataset).
- out_and_tx_malicious: 1 if tx_hash is a malicious transaction or an output of a malicious transaction.
- all_malicious: 1 if tx_hash is a malicious transaction or an output/input of a malicious transaction.


Connect Gdrive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import all necessary Modules

In [3]:
import os#file operations
import numpy as np#linear algebra
import pandas as pd#data processing, CSV I/O
from matplotlib.colors import LinearSegmentedColormap#custom colormaps
import matplotlib.pyplot as plt#plotting
import matplotlib.dates as mdates#date formatting
import matplotlib.colors as mcolors#color utilities
import seaborn as sns#statistical data visualization
import plotly.graph_objects as go#interactive plots
import plotly.express as px#high-level plotting
import networkx as nx#graph/network analysis
from sklearn.preprocessing import RobustScaler
from scipy import stats
pd.set_option('display.max_columns', None)#display all columns
import missingno as msno

Check which data files are downloaded.

In [4]:
def data_paths(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            print(os.path.join(root, file))

directory_path = '/content/drive/MyDrive/BlockChainAnomaly/Data'
data_paths(directory_path)

/content/drive/MyDrive/BlockChainAnomaly/Data/unmatched_transactions.csv
/content/drive/MyDrive/BlockChainAnomaly/Data/transaction_dataset.csv
/content/drive/MyDrive/BlockChainAnomaly/Data/._DG_out.csv
/content/drive/MyDrive/BlockChainAnomaly/Data/DG_out.csv
/content/drive/MyDrive/BlockChainAnomaly/Data/txs_features.csv
/content/drive/MyDrive/BlockChainAnomaly/Data/wallets_classes.csv
/content/drive/MyDrive/BlockChainAnomaly/Data/wallets_features_classes_combined.csv
/content/drive/MyDrive/BlockChainAnomaly/Data/wallets_features.csv
/content/drive/MyDrive/BlockChainAnomaly/Data/tabnet_Ethereum_model.zip
/content/drive/MyDrive/BlockChainAnomaly/Data/nn_wallet_model.h5
/content/drive/MyDrive/BlockChainAnomaly/Data/cnn_wallet_history.pkl
/content/drive/MyDrive/BlockChainAnomaly/Data/tabnet_wallet_loss_history.npy
/content/drive/MyDrive/BlockChainAnomaly/Data/tabnet_wallet_model.zip
/content/drive/MyDrive/BlockChainAnomaly/Data/cnn_wallet_model.h5
/content/drive/MyDrive/BlockChainAnomaly/D

Load Transactions Metadata

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/BlockChainAnomaly/Data/DG_out.csv', encoding='utf-8')
print(df.head())


Load Hackfiles

In [ ]:
hack_files = {
    'anomalies_loss': '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/other/anomalies_loss_tx.csv',
    'anomalies_misc': '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/other/anomalies_misc_tx.csv',
    'anomalies_seizure': (
        '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/other/anomalies_seizure1_tx.csv',
        '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/other/anomalies_seizure2_tx.csv'
    ),
    'anomalies_theft': '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/other/anomalies_theft_tx.csv',
    'malicious_tx_out': '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/malicious_tx_out.csv',
    'malicious_tx_in': '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/malicious_tx_in.csv'
}

In [ ]:
# Initialize the DataFrame and counters
unmatched_tx = pd.DataFrame(columns=['tx_hash', 'type'])
total_transactions = 0
total_matched = 0
total_unmatched = 0

In [ ]:
print(df.columns)


In [ ]:

# Define hack_files as provided
hack_files = {
    'anomalies_loss': '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/other/anomalies_loss_tx.csv',
    'anomalies_misc': '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/other/anomalies_misc_tx.csv',
    'anomalies_seizure': (
        '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/other/anomalies_seizure1_tx.csv',
        '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/other/anomalies_seizure2_tx.csv'
    ),
    'anomalies_theft': '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/other/anomalies_theft_tx.csv',
    'malicious_tx_out': '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/malicious_tx_out.csv',
    'malicious_tx_in': '/content/drive/MyDrive/BlockChainAnomaly/Data/bitcoin_hacks_2010to2013/malicious_tx_in.csv'
}

# Initialize DataFrame and counters
unmatched_tx = pd.DataFrame(columns=['tx_hash', 'type'])
total_transactions = 0
total_matched = 0
total_unmatched = 0

# Loop through each hack file
for hack_type, file_path in hack_files.items():
    if isinstance(file_path, tuple):
        # Concatenate data if multiple files are present
        tx_hashes = pd.concat([pd.read_csv(f, header=None) for f in file_path])[0]
    else:
        # Read a single file
        tx_hashes = pd.read_csv(file_path, header=None)[0]

    total_transactions += len(tx_hashes)
    tx_hashes_set = set(tx_hashes)

    # Add a new column to df to mark matched transactions
    df[hack_type] = df['tx_hash'].isin(tx_hashes_set).astype(int)
    matched_count = df[hack_type].sum()
    total_matched += matched_count

    # Identify unmatched transactions
    unmatched_hashes = tx_hashes_set - set(df['tx_hash'])
    unmatched_count = len(unmatched_hashes)
    total_unmatched += unmatched_count

    # Add unmatched transactions to unmatched_tx DataFrame
    unmatched_df = pd.DataFrame({'tx_hash': list(unmatched_hashes), 'type': hack_type})
    unmatched_tx = pd.concat([unmatched_tx, unmatched_df], ignore_index=True)

    # Print results for each hack type
    print(f"Results for {hack_type}:")
    print(f"Total transactions in hack file: {len(tx_hashes)}")
    print(f"Matched transactions: {matched_count}")
    print(f"Unmatched transactions: {unmatched_count}")
    print("-" * 50)

# Optionally, print overall results
print("Overall Results:")
print(f"Total transactions: {total_transactions}")
print(f"Total matched transactions: {total_matched}")
print(f"Total unmatched transactions: {total_unmatched}")


In [ ]:
print("Overall Results:")
print(f"Total transactions processed: {total_transactions}")
print(f"Total matched transactions: {total_matched}")
print(f"Total unmatched transactions: {total_unmatched}")

Save Unmatched Transactions

In [ ]:
unmatched_tx.to_csv('/content/drive/MyDrive/Azizah_BlockChainAnomalyProject/Data/unmatched_transactions.csv', index=False)

Analysis

In [ ]:
df.info()

30,248,134 entries with 19 columns

In [ ]:
df.describe()

Summary shows that the dataset contains entries with a significant variance in transaction amounts and connectivity. The average values for indegree and outdegree are around 2.15, while the average transaction amounts (in_btc and out_btc) are approximately 54.84 BTC. Malicious activities and anomalies are rare, as indicated by their mean values being close to zero. The maximum values for bitcoin transactions are extremely high, reaching up to 550,000 BTC, highlighting the presence of outliers.

In [ ]:
missing_values = df.isnull().sum()
print(missing_values)
msno.matrix(df)

In [ ]:
malicious_columns = [
    'is_malicious', 'in_malicious', 'out_malicious',
    'out_and_tx_malicious', 'anomalies_loss',
    'anomalies_misc', 'anomalies_seizure'
]
malicious_counts = df[malicious_columns].sum()
malicious_counts_df = pd.DataFrame({'Category': malicious_counts.index, 'Count': malicious_counts.values})
malicious_counts_df = malicious_counts_df.sort_values(by='Count', ascending=False)
print(malicious_counts_df)
#plot
plt.figure(figsize=(13, 8))
sns.barplot(x='Category', y='Count', data=malicious_counts_df)
plt.title('Counts of Different Types of Malicious Transactions')
plt.xticks(rotation=45)
plt.ylabel('Count')
plt.xlabel('Malicious Categories')
plt.show()

Thismeans that incoming malicious transactions and seizure anomalies are the most frequent, while other types of malicious activities and anomalies are relatively rare in the dataset. Out malicious are only 65 in dataset.

In [ ]:
malicious_df = df[malicious_columns]
corr = malicious_df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap of Malicious Categories')
plt.show()

The strong positive correlation between is_malicious and out_and_tx_malicious suggests that when a transaction is identified as malicious, it is likely to also be an output or directly related to another malicious transaction.

In [ ]:
malicious_counts = df[malicious_columns].sum()
malicious_counts_percentage = (malicious_counts / malicious_counts.sum()) * 100
fig, ax = plt.subplots(figsize=(10, 10))
wedges, texts, autotexts = ax.pie(
    malicious_counts_percentage,
    labels=malicious_counts.index,
    autopct='%1.1f%%',
    startangle=140,
    colors=sns.color_palette("coolwarm", len(malicious_counts)),
    wedgeprops={'edgecolor': 'black'}
)

ax.legend(wedges, malicious_counts.index, title="Categories", loc="upper left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.setp(autotexts, size=10, weight="bold", color="white")
plt.setp(texts, size=12)
ax.set_title('Distribution of Malicious Transactions', fontsize=15)
plt.ylabel('')
plt.show()

- in_malicious: 49.5% of transactions are inputs to malicious accounts.
- anomalies_seizure: 40.5% of transactions involve seizure-related anomalies.
- out_and_tx_malicious: 4.4% of transactions are both outputs and linked to malicious activities.
- out_malicious: 2.6% of transactions are endpoints or dispersal points of malicious funds.
- is_malicious: 1.9% of transactions are identified as generally malicious.
- anomalies_loss: 0.8% of transactions report loss anomalies.
- anomalies_misc: Less than 0.1% of transactions are miscellaneous anomalies.

In [ ]:
transaction_features = ['indegree', 'outdegree', 'in_btc', 'out_btc', 'total_btc', 'mean_in_btc', 'mean_out_btc']
malicious_indicators = ['in_malicious', 'out_malicious', 'is_malicious', 'out_and_tx_malicious', 'all_malicious',
                        'anomalies_loss', 'anomalies_misc', 'anomalies_seizure']
heatmap_df = df[transaction_features + malicious_indicators]
correlation_matrix = heatmap_df.corr()
plt.figure(figsize=(14, 10))
heatmap = sns.heatmap(correlation_matrix, annot=True, cmap='viridis', fmt='.2f', linewidths=.5, cbar_kws={"shrink": .75})
plt.title('Correlation Heatmap: Transaction Features vs. Malicious Indicators', fontsize=16, weight='bold', pad=20)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(rotation=0, fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
cols = ['indegree', 'outdegree', 'in_btc', 'out_btc', 'total_btc', 'mean_in_btc', 'mean_out_btc']
df[cols].hist(figsize=(18, 20), bins=50, xlabelsize=9, ylabelsize=9)
plt.suptitle('Original Data Distributions', fontsize=16)
plt.show()

Logged Scale

In [ ]:
log_df = np.log(df[cols].replace(0, np.nan)).replace(-np.inf, -1).fillna(0)

#logged data distributions
log_df.hist(figsize=(20, 20), bins=50, xlabelsize=9, ylabelsize=9)
plt.suptitle('Log-Transformed Data Distributions', fontsize=16)
plt.show()

Scaled Logged data

In [ ]:
rob_scaler = RobustScaler()

scaled_df = log_df.copy()
scaled_df[cols] = rob_scaler.fit_transform(log_df[cols])

#scaled logged data distributions
scaled_df.hist(figsize=(20, 20), bins=50, xlabelsize=9, ylabelsize=9)
plt.suptitle('Scaled Log-Transformed Data Distributions', fontsize=16)
plt.show()

Scaling on logged data has very efficient Response to the Distribution of variables, so in modelling, It must be considred.